In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
from pymongo import MongoClient
from pprint import pprint 


## Saving prices to database

In [ ]:
client = MongoClient('localhost', 27017)
db = client['crypto']
collection = db['btcusdt_price']

if (collection.count() != 0):
    collection.drop()

DATA = './data/BTCUSDT.csv'
df = pd.read_csv(DATA)
data = df.to_dict(orient='recoreds')

# inserting data to collection['btcusdt_price']
collection.insert_many(data)

# reading data from collection['btcusdt_price'] without id 
df_from_db = pd.DataFrame(list(collection.find({}, {'_id': 0})))
# df_from_db = pd.DataFrame(list(collection.find()))

## Graph Function 

In [98]:
import plotly.express as px
import plotly.graph_objects as go 
from plotly.subplots import make_subplots

def linegraph(x_data, y_data, title):
    fig = make_subplots(rows=1, cols=1)
    fig.add_trace(go.Bar(x=x_data, y=y_data,  name=title))
    fig.update_layout(height=1000, width=1500)
    fig.show()


def multilinegraph(x_data, y1_data, y2_data, title, graph_mode):
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(go.Scatter(x=x_data, y=y1_data, mode=graph_mode, name=title[0]), secondary_y=False)
    fig.add_trace(go.Scatter(x=x_data, y=y2_data, mode=graph_mode, name=title[1]), secondary_y=True)

   
    # fig.add_trace(go.Bar(x=df_inflow_big.datetime, y=df_price_change.price_change,  name='price'), secondary_y=False)
    # fig.add_trace(go.Bar(x=df_inflow_big.datetime, y=df_inflow_big.inflow_total, name='inflow'), secondary_y=True)
    fig.update_layout(height=1000, width=1500)
    fig.show()

# fig = make_subplots(specs=[[{"secondary_y": True}]])
# # fig.add_trace(go.Scatter(x=df_inflow_big.datetime, y=df_price_change.price_change, mode='markers', name='price'), secondary_y=False)
# # fig.add_trace(go.Scatter(x=df_inflow_big.datetime, y=df_inflow_big.inflow_total, mode='markers', name='inflow'), secondary_y=True)
# fig.add_trace(go.Bar(x=df_inflow_big.datetime, y=df_price_change.price_change,  name='price'), secondary_y=False)
# fig.add_trace(go.Bar(x=df_inflow_big.datetime, y=df_inflow_big.inflow_total, name='inflow'), secondary_y=True)
# fig.update_layout(height=1000, width=1500)
# fig.show()

def multibargraph(x_data, y1_data, y2_data, title):
    fig = go.Figure(data=[
            go.Bar(x=x_data, y=y1_data, name=title[0]),
            go.Bar(x=x_data, y=y2_data, name=title[1])
    ])
    # fig.add_trace(go.Bar(x=x_data, y=y1_data, name=title[0]), secondary_y=False)
    # fig.add_trace(go.Bar(x=x_data, y=y2_data, name=title[1]), secondary_y=True)

   
    # fig.add_trace(go.Bar(x=df_inflow_big.datetime, y=df_price_change.price_change,  name='price'), secondary_y=False)
    # fig.add_trace(go.Bar(x=df_inflow_big.datetime, y=df_inflow_big.inflow_total, name='inflow'), secondary_y=True)
    fig.update_layout(height=1000, width=1500, barmode='group')
    fig.show()

## Experiments on CryptoQuant Data 

In [5]:
import requests
from pprint import pprint 
import configparser
from urllib.parse import urljoin 
from pprint import pprint 
from datetime import datetime


config = configparser.ConfigParser()
config.read('config.ini')

API_KEY = config['CRYPTOQUANT']['API_KEY']
API_URL = config['CRYPTOQUANT']['API_URL']

headers = {'Authorization': 'Bearer ' + API_KEY}

### Coinbase Premium Data Generation

In [14]:
PATH = 'btc/market-data/coinbase-premium-index'
URL = urljoin(API_URL, PATH)
params = {
    'window': 'min',
    'limit': 100000
}

response = requests.get(URL, headers=headers, params=params).json()
# print(response['status'])
coinbase_premium_index = response['result']['data']
len(coinbase_premium_index)

df_coinbase = pd.DataFrame(coinbase_premium_index)

In [19]:
day_range = 1500

df_coinbase = df_coinbase[:day_range]


linegraph(x_data=df_coinbase.datetime, y_data=df_coinbase.coinbase_premium_gap, title='premium_index')

In [21]:
from numpy import linspace
max_gap = df_coinbase.coinbase_premium_gap.max()
min_gap = df_coinbase.coinbase_premium_gap.min()

quarters = linspace(min_gap, max_gap, 4)
quarters

array([-187.88      ,  -91.12666667,    5.62666667,  102.38      ])

In [22]:

in_date = df_coinbase.loc[0].datetime
d = datetime.strptime(in_date, '%Y-%m-%d %H:%M:%S')
new_date = d.strftime('%Y%m%dT%H%M%S')

print(new_date)

20210414T020400


In [23]:
PATH = 'btc/market-data/price-usd'
URL = urljoin(API_URL, PATH)
params = {
    'window': 'min',
    'to': new_date,
    'limit': 100000
}

response = requests.get(URL, headers=headers, params=params).json()
price_usd = response['result']['data']
df_price = pd.DataFrame(price_usd)

In [117]:
df_price = df_price[:day_range]
df_price

max_price = df_price.price_usd_close.max()
min_price = df_price.price_usd_close.min()


In [92]:
multilinegraph(x_data=df_coinbase.datetime, y1_data=df_coinbase.coinbase_premium_gap, y2_data=df_price.price_usd_close, title=['premium_index', 'price'], graph_mode='markers')

In [123]:
size = len(df_price)
size

1500

In [126]:

# y1_min = df_coinbase
# y1_max = 

# y2_min = 
# y2_max = 




interval = 1500

import plotly.graph_objects as px
plot = px.Figure(data=[
        go.Bar(x=df_coinbase.datetime[:interval], y=df_coinbase.coinbase_premium_gap[:interval], yaxis='y', offsetgroup=1, name='premium'),
        go.Bar(x=df_coinbase.datetime[:interval], y=df_price.price_usd_close[:interval], yaxis='y2', offsetgroup=2, name='price')
        # go.Bar(x=df_coinbase.datetime[:50], y=df_coinbase.coinbase_premium_gap[:50], yaxis='y', offsetgroup=1, name='premium'),
        # go.Bar(x=df_coinbase.datetime[:50], y=df_price.price_usd_close[:50], yaxis='y2', offsetgroup=2, name='price')
    ],
    
    layout={
        'yaxis': {'title': 'premium'},
        'yaxis2': {'title': 'price', 'overlaying':'y', 'side': 'right'},
    })

plot.update_layout(height=1000, width=4000, barmode='group', yaxis=dict(range=[min_gap, max_gap]), yaxis2=dict(range=[min_price, max_price]))       
plot.show()

In [59]:
quarters

df_coinbase[df_coinbase.coinbase_premium_gap < quarters[0]]

# df_coinbase[df_coinbase.coinbase_premium_gap > quarters[1]]

,datetime,coinbase_premium_gap,coinbase_premium_index


In [71]:
import numpy as np


df_test0 = df_coinbase[df_coinbase.coinbase_premium_gap <= quarters[1]]
df_test0

df_test1 = df_coinbase[(df_coinbase.coinbase_premium_gap > quarters[1]) & (df_coinbase.coinbase_premium_gap <= quarters[2])]
df_test1

df_test2 = df_coinbase[(df_coinbase.coinbase_premium_gap > quarters[2]) & (df_coinbase.coinbase_premium_gap <= quarters[3])]
df_test2


len(df_test0) + len(df_test1) + len(df_test2) 
print(len(df_test0))
print(len(df_test1))
print(len(df_test2))


print(len(df_test0) + len(df_test1) + len(df_test2))


35
741
724
1500


In [87]:
import numpy as np
# df_coinbase['coinbase_premium_gap'] < quarters[1] or df_coinbase['coinbase_premium_gap'] > quarters[3]

# df_coinbase_outliers_index = df_coinbase.query([(df_coinbase.coinbase_premium_gap <= quarters[1]) | (df_coinbase.coinbase_premium_gap > quarters[2])]).index.tolist()
df_coinbase_outliers = df_coinbase[(df_coinbase.coinbase_premium_gap <= quarters[1]) | (df_coinbase.coinbase_premium_gap > quarters[2])]
 

len(df_coinbase_outliers)

# index = np.where((df_coinbase.coinbase_premium_gap < quarters[1]) | (df_coinbase.coinbase_premium_gap > quarters[3]), True, False)
# index = df_coinbase[df_coinbase.coinbase_preminum_gap < quarters[1]]
# index

759

In [75]:
linegraph(x_data=df_coinbase_outliers.datetime, y_data=df_coinbase_outliers.coinbase_premium_gap, title='premium')


In [89]:
df_coinbase_outliers.index

df_price_outliers = df_price.iloc[df_coinbase_outliers.index ]


multilinegraph(x_data=df_coinbase_outliers.datetime, y1_data=df_coinbase_outliers.coinbase_premium_gap, y2_data=df_price_outliers.price_usd_close, title=['premium','price'])


In [ ]:
PATH = 'btc/exchange-flows/inflow'
URL = urljoin(API_URL, PATH)
params = {
    'exchange': 'all_exchange',
    'window': 'block',
    'limit': 100000,
    'to': 677958

}

response = requests.get(URL, headers=headers, params=params).json()
# print(response['status'])
inflow_all_exchange = response['result']['data']
len(inflow_all_exchange)



PATH = 'btc/exchange-flows/inflow'
URL = urljoin(API_URL, PATH)
params = {
    'exchange': 'binance',
    'window': 'block',
    'limit': 300000,
    'to': 677958

}

response = requests.get(URL, headers=headers, params=params).json()
# print(response['status'])
inflow_binance = response['result']['data']
len(inflow_binance)



PATH = 'btc/market-data/price-usd'
URL = urljoin(API_URL, PATH)
params = {
    'window': 'block',
    'limit': 100000,
    'to': 677958
}

response = requests.get(URL, headers=headers, params=params).json()
price = response['result']['data']
len(price)

In [ ]:
import pandas as pd
df_price = pd.DataFrame(price).sort_values(by=['blockheight'], ascending=True)
df_price = df_price.reset_index(drop=True)

# last two elements 
df_price[-2:] 
# df_price
# print(pd.isnull(df_price).sum())


In [ ]:
df_inflow_all_exchange = pd.DataFrame(inflow_all_exchange).sort_values(by=['blockheight'],ascending=True)
df_inflow_all_exchange = df_inflow_all_exchange.reset_index(drop=True)
df_inflow_all_exchange[-2:]
# print(pd.isnull(df_inflow).sum())

In [ ]:
df_price['blockheight'] == df_inflow_all_exchange['blockheight']
df_price['datetime'] == df_inflow_all_exchange['datetime']

### Correlation between price and inflow_total

In [ ]:
import numpy as np 

price_change = df_price['price_usd_close'].diff().fillna(0)
price_change

x= price_change
y = df_inflow_all_exchange['inflow_total']
print(np.corrcoef(x, y))

x= df_price['price_usd_close']
y = df_inflow_all_exchange['inflow_total']
print(np.corrcoef(x, y))

In [ ]:
df_price

In [ ]:
inflow_threshold = 1000
# filter indexes with inflow > 1000 
inflow_big = df_inflow_all_exchange.index[df_inflow_all_exchange['inflow_total'] >= inflow_threshold]


# def subset_sum(item, offset):
#     #  print('X item: ', item)
#     subsum = test_price['price_usd_close'].iloc[item.name+1: item.name+offset].sum() 
#     print(subsum)
#     print('item: ', item.name)
#     if (np.isnan(subsum)):
#         print('item: ', item.name)
#         print('price: ', item['price_usd_close'])
#     return(subsum)

def subset_sum(item, offset):
     return df_price['price_usd_close'].iloc[item.name+1: item.name+offset].mean() - item['price_usd_close']

# def subset_sum(start_index, offset):
#      return df_price['price_usd_close'].iloc[start_index+1: start_index+offset].sum()

def inflow_to_price_change(df_price, offset, inflow_big_indexes): 
    return df_price.apply(lambda x: subset_sum(x, offset) if (x.name in inflow_big_indexes) else 0 , axis=1)
    
# def inflow_to_price_change(df_price, offset, inflow_big_indexes): 
#    return df_price.apply(lambda x: subset_sum(x.name, offset) if (x.name in inflow_big_indexes) else 0 , axis=1)

offset = 10 
price_change = inflow_to_price_change(df_price, offset, inflow_big)

price_change
# price_change.iloc[32]



In [ ]:
price_change
type(inflow_big)

df_price_change = pd.DataFrame(price_change.iloc[inflow_big], columns=['price_change']) 
df_price_change

In [ ]:
df_inflow_big = df_inflow_all_exchange[df_inflow_all_exchange['inflow_total'] >= inflow_threshold]
df_inflow_big

df_price_change = pd.DataFrame(price_change.iloc[inflow_big], columns=['price_change']) 
df_price_change

In [ ]:
df_inflow_big = df_inflow_big[:30]
df_price_change = df_price_change[:30]

In [ ]:
import plotly.express as px
import plotly.graph_objects as go 
from plotly.subplots import make_subplots

fig = make_subplots(specs=[[{"secondary_y": True}]])
# fig.add_trace(go.Scatter(x=df_inflow_big.datetime, y=df_price_change.price_change, mode='markers', name='price'), secondary_y=False)
# fig.add_trace(go.Scatter(x=df_inflow_big.datetime, y=df_inflow_big.inflow_total, mode='markers', name='inflow'), secondary_y=True)
fig.add_trace(go.Bar(x=df_inflow_big.datetime, y=df_price_change.price_change,  name='price'), secondary_y=False)
fig.add_trace(go.Bar(x=df_inflow_big.datetime, y=df_inflow_big.inflow_total, name='inflow'), secondary_y=True)
fig.update_layout(height=1000, width=1500)
fig.show()
# e2 = go.Scatter(x=df_price_and_inflow.datetime_x, y=df_price_and_inflow.inflow_total, mode='lines')
# |ip# lot(go.Figure([trace1, trace2]))


In [ ]:
df = df_price.copy()

df['price_change'] = pd.DataFrame(price_change)
df
# df[['price_change', 'infflow_total']]  = pd.DataFrame(price_change), df_inflow_all_exchange['inflow_total']

# df1 = df_inflow_all_exchange['inflow_total']

In [ ]:
df_inflow_all_exchange['inflow_total']

In [ ]:
df_price['price_change'].iloc[32]

In [ ]:
df_price_and_inflow_all_exchange = pd.merge(df_price, df_inflow_all_exchange, on='blockheight')
# df_price_and_inflow
df_price_and_inflow_all_exchange[['datetime_x', 'price_usd_close', 'inflow_total']]




In [ ]:
df_data = df_price_and_inflow_all_exchange[-500:]
df_data

In [ ]:
from sklearn.preprocessing import StandardScaler

df_data[['norm_inflow']] = StandardScaler().fit_transform(df_data[['inflow_total']])
df_data['norm_price'] = df_data['price_usd_close']/10000
# df_data[['norm_price', 'norm_inflow']] = StandardScaler().fit_transform(df_data[['price_usd_close', 'inflow_total']])


In [ ]:
df_data

In [ ]:
price_change = df_data['price_usd_close'].diff().fillna(0)
price_change


In [ ]:
df_data

## Test: Inflow changes Price?

In [ ]:
import plotly.express as px
import plotly.graph_objects as go 


from plotly.offline import init_notebook_mode, iplot

from plotly.subplots import make_subplots



fig = make_subplots(specs=[[{"secondary_y": True}]])# 
afig.add_trace(go.Scatter(x=df_data.datetime_x, y=price_change, name='price'), secondary_y=Falsee
1fig.add_trace(go.Scatter(x=df_data.datetime_x, y=df_data.inflow_total, name='inflow'), secondary_y=True)
ffig.update_layout(height=1000, width=1500)ig.show()e2 = go.Scatter(x=df_price_and_inflow.datetime_x, y=df_price_and_inflow.inflow_total, mode='lines')
ip# lot(go.Figure([trace1, trace2]))

ig =o.Fi#
 gure()a
fig = px.li# ne(df_price_df_datatime_x', y=['price_usdnorm_pricew_tonorm_inflow='Prheight=1000, width=1500w 

# # fig = px.line(df_data, x='datetime_x', y=['price_usd_close', 'inflow_total'], height=1000, width=1500) 
r
#fi# g = make_subplots(specs=[[{'secondary_y': True}]])
f
fig# fig = pline(df_price_and_inflow, x='datetime_x', y='price_usd_close', title='Price')
a# dd_scatter(cq_idf_price_and_inflow'datetime', y_x='inflow_total', title='Inflo')ea_c


h
fig =#  px._price_and_inflow='datetime', _xy='infprice_usd_closetitle='InfPrice Changear
earea_char# t = px.area(df_price_and_inflow, x='datetime_x', y='inflow_total', title='Price Change')_chart.upda# te_xaxes(title_text = 'Date')
area_chart.updat# e_yaxes(title_text = 'Infl'Price Change'kprefix = '$')
area_chart.update# _layout(showlegend = False)
area_chart.show()# 

In [ ]:
df_data['price_usd_close'].diff()

type(df_data['price_usd_close'].diff())

In [ ]:
import numpy as np 

x= price_change
y = df_data['inflow_total']
print(np.corrcoef(x, y))


x= df_data['price_usd_close']
y = df_data['inflow_total']
print(np.corrcoef(x, y))

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=[1, 2, 3], y=[40, 50, 60], name="yaxis data"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=[2, 3, 4], y=[4, 5, 6], name="yaxis2 data"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Double Y Axis Example"
)

# Set x-axis title
fig.update_xaxes(title_text="xaxis title")

# Set y-axes titles
fig.update_yaxes(title_text="<b>primary</b> yaxis title", secondary_y=False)
fig.update_yaxes(title_text="<b>secondary</b> yaxis title", secondary_y=True)

fig.show()

In [ ]:
#fig = make_subplots(specs=[[{'secondary_y': True}]])
fig = go.Figure()

fig = px.line(cq_price, x='datetime', y=['price_usd_close', 'inflow_total'], title='Price')
# fig.add_scatter(cq_inflow, x='datetime', y='inflow_total', title='Inflo')


# fig.add_trace(go.Scatter(x=cq_price['datetime'], y=cq_prce['price_usd_close']),  marker=dict(color="blue"),showlegend=False)
# fig.add_trace(go.Scatter(x=cq_price['datetime'], y=cq_inflow['inflow_total']),  marker=dict(color="green"),showlegend=False)

fig.show()


In [ ]:
trace1 = go.Scatter(x=df_price_and_inflow.datetime_x, y=df_price_and_inflow.price_usd_close, mode='lines')
trace2 = go.Scatter(x=df_price_and_inflow.datetime_x, y=df_price_and_inflow.inflow_total, mode='lines')
iplot(go.Figure([trace1, trace2]))

In [ ]:
cq_data[0:5]

In [ ]:
PATH = 'btc/exchange-flows/reserve'
URL = urljoin(API_URL, PATH)
params = {
    'exchange': 'binance',
    'window': 'block',
    'limit': 100000
}

response = requests.get(URL, headers=headers, params=params).json()
# print(response['status'])
cq_data = response['result']['data']
len(cq_data)

In [ ]:
cq_data[0:5]

In [ ]:
PATH = 'btc/exchange-flows/outflow'
URL = urljoin(API_URL, PATH)
params = {
    'exchange': 'binance',
    'window': 'block',
    'limit': 100000
}

response = requests.get(URL, headers=headers, params=params).json()
# print(response['status'])
cq_data = response['result']['data']
len(cq_data)

In [ ]:
df_cq_data = pd.DataFrame(cq_data)

In [ ]:
# df_cq_data['datetime']
df_cq_data[0:5]

In [ ]:
import plotly.express as px
import plotly.graph_objects as go 

area_chart = px.area(df_cq_data, x='datetime', y='inflow_total', title='Inflow Total')
area_chart.update_xaxes(title_text = 'Date')
area_chart.update_yaxes(title_text = 'Inflow Total', tickprefix = '$')
area_chart.update_layout(showlegend = False)
area_chart.show()



In [ ]:
candlestick = go.Figure(data = [go.Candlestick(x = df_from_db['Date'], 
                                               open = df_from_db['Open'], 
                                               high = df_from_db['High'], 
                                               low = df_from_db['Low'], 
                                               close = df_from_db['Close'])])

candlestick.update_layout(xaxis_rangeslider_visible = False, title = 'BTCUSDT')
candlestick.update_xaxes(title_text = 'Date')
candlestick.update_yaxes(title_text = 'BTCUSDT Close Price', tickprefix = '$')

candlestick.show()

In [ ]:
df['Close']

In [ ]:
df['Close'].plot()

In [ ]:
max(df['Close'])

In [ ]:
min(df['Close'])

In [ ]:
df['Close'].rolling(3).mean()



In [ ]:
fig, ax = plt.subplots(figsize=(10,5))

ax.set_title('INDEX', fontsize=15)
ax.set_ylabel("BTCUSDT")
ax.set_xlabel("Date Time")
ax.plot(df.index, df[['Close'], df['Close'].rolling(3).mean])
ax.legend(['Close','MA5','MA10'])
plt.show()